## 7.1.2 컨투어 단순화
- approx = cv2.approxPolyDP(contour, epsilon, closed)
    - contour : 대상 컨투어 좌표
    - epsilon : 근사 값 정확도, 오차 범위
    - closed : 컨투어의 닫힘 여부
    - approx : 근사 계산한 컨투어 좌표

In [15]:
import cv2
import numpy as np

img1 = cv2.imread('img/bad_rect.png')
img2 = img1.copy()

# to gray scale
img_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# 스레시홀드 바이너리 이미지로 만들어 검은 배경에 흰 전경으로 반전
ret, imthres = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY_INV)

# find contour
contours, hierarchy = cv2.findContours(imthres, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

contour = contours[0]
# 전체 둘레의 0.05로 오차범위 지정
epsilon = 0.05*cv2.arcLength(contour, True)
# approx contour
approx = cv2.approxPolyDP(contour, epsilon, True)

# 각각 컨투어 선 그리기
cv2.drawContours(img1, [contour], -1, (0, 255, 0), 3)
cv2.drawContours(img2, [approx], -1, (0, 255, 0), 3)

# result
cv2.imshow('contour', img1)
cv2.imshow('approx', img2)
cv2.waitKey()
cv2.destroyAllWindows()

- hull = cv2.convexHull(points[, hull, clockwise, returnPoints]) : 블록 선체 찾기
    - points : 입력 컨투어
    - hull : 볼록 선체 결과
    - clockwise : 방향 지정 (True : 시계방향)
    - returnPoints : 결과 좌표 형식 선택 (True: 볼록 선체 좌표 반환 / False: 입력 컨투어 중에 볼록 선체에 해당하는 인덱스 반환)
- retval = cv2.isContourConvex(contour) : 볼록 선체 만족 여부 확인
    - retval : True인 경우 볼록 선체 만족
- [start, end, farthest, distance] = cv2.convexityDefects(contour, convexhull) : 볼록 선체 결함 찾기
    - contour : 입력 컨투어
    - convexhull : 볼록 선체에 해당하는 컨투어의 인덱스
    - defects : 볼록 선체 결함이 있는 컨투어의 배열 인덱스 (N x 1 x 4 배열)
    - defects : [start, end, farthest, distance]
        - start : 오목한 각이 시작되는 컨투어의 인덱스
        - end : 오목한 각이 끝나는 컨투어의 인덱스
        - farthest : 볼록 선체에서 가장 먼 오목한 지점의 컨투어 인덱스
        - distance : farthest와 볼록 선체와의 거리 (8비트 고정 소수점, distance/256.0)

In [18]:
import cv2
import numpy as np

img1 = cv2.imread('img/hand.jpg')
img2 = img1.copy()

# to gray scale
img_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# 스레시홀드 바이너리 이미지로 만들어 검은 배경에 흰 전경으로 반전
ret, imthres = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY_INV)

# find contour
contours, hierarchy = cv2.findContours(imthres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contour = contours[0]
cv2.drawContours(img1, [contour], -1, (0, 255, 0), 1)

# 볼록선체 찾기 (좌표기준) & 그리기
hull = cv2.convexHull(contour)
cv2.drawContours(img2, [hull], -1, (0, 255, 0), 1)
print(cv2.isContourConvex(contour), cv2.isContourConvex(hull))

# 볼록선체 찾기 (인덱스 기준)
hull2 = cv2.convexHull(contour, returnPoints=False)
# 볼록선체 결함 찾기
defects = cv2.convexityDefects(contour, hull2)
# 볼록 선체 결함 순회
for i in range(defects.shape[0]):
    # 시작, 종료, 가장 먼 지점, 거리
    startP, endP, farthestP, distance = defects[i, 0]
    # 가장 먼 지점 좌표
    farthest = tuple(contour[farthestP][0])
    # 거리를 부동 소수점으로 변환
    dist = distance/256.0
    # 거리가 1보다 큰 경우, red dot marker
    if dist > 1:
        cv2.circle(img2, farthest, 3, (0, 0, 255), -1)

# result
cv2.imshow('contour', img1)
cv2.imshow('convex hull', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

False True


### 7.1.3 컨투어와 도형 매칭
- retval = cv2.matchShapes(contour1, contour2, method, parameter) : 두 개의 컨투어로 도형 매칭
    - method : 휴 모멘트 비교 알고리즘 선택 플래그
        - cv2.CONTOURS_MATCH_I1
        - cv2.CONTOURS_MATCH_I2
        - cv2.CONTOURS_MATCH_I3
    - parameter : 알고리즘에 전달을 위한 예비 인수, 현재는 지원안됨 (0으로 고정)
    - retval : 닮은 정도, 0 = 동일, 클수록 다름

In [20]:
import cv2
import numpy as np

target = cv2.imread('img/4star.jpg')
shapes = cv2.imread('img/shapestomatch.jpg')

# to gray scale
target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
shapes_gray = cv2.cvtColor(shapes, cv2.COLOR_BGR2GRAY)
# 스레시홀드 바이너리 이미지로 만들어 검은 배경에 흰 전경으로 반전
ret, targetTh = cv2.threshold(target_gray, 127, 255, cv2.THRESH_BINARY_INV)
ret, shapeTh = cv2.threshold(shapes_gray, 127, 255, cv2.THRESH_BINARY_INV)

# find contour
target_contours, _ = cv2.findContours(targetTh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
shape_contours, _ = cv2.findContours(shapeTh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 각 도형과 매칭을 위한 반복문
matches = []
for contour in shape_contours:
    match = cv2.matchShapes(target_contours[0], contour, cv2.CONTOURS_MATCH_I2, 0.0)
    matches.append((match, contour))
    cv2.putText(shapes, '%.2f' % match, tuple(contour[0][0]), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)

# 매칭 점수로 정렬
matches.sort(key=lambda x : x[0])


# result : 가장 낮은 점수 얻는 도형에 컨투어 그리기
cv2.drawContours(shapes, [matches[0][1]], -1, (0, 255, 0), 3)
cv2.imshow('target', target)
cv2.imshow('Match shapes', shapes)
cv2.waitKey()
cv2.destroyAllWindows()

## 7.2 허프 변환 (Hough transform)
- 직선을 찾는 방법으로 시작했다가 다양한 모양을 인식하게 확장

### 7.2.1 허프선 변환
- 직선 관계를 갖는 픽셀들만 골라내는 것이 핵심
- lines = cv2.HoughLines(image, rho, theta, threshold[, lines, srn=0, stn=0, min_theta, max_theta])
    - rho : 거리 측정 해상도, 0~1
    - theta : 각도 측정 해상도, 0~1
    - threshold : 직선으로 판단할 최소한의 동일 개수
        - 작은 값 : 정확도 감소, 검출 개수 증가
        - 큰 값 : 정확도 증가, 검출 개수 감소
    - lines : 검출 결과 (N x 1 x 2) 배열 (r , theta)
    - srn, stn : 멀티 스케일 허프 변환에 사용, 선 검출에서는 사용 안함
    - min_theta, max_theta : 검출을 위해 사용할 최대, 최소 각도

In [28]:
import cv2
import numpy as np

img1 = cv2.imread('img/sudoku.jpg')
img2 = img1.copy()
h, w = img1.shape[:2]

# to gray scale
img_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(img_gray, 100, 200)

# find Hough line
lines = cv2.HoughLines(edges, 1, np.pi/180, 130)
# print(lines)

for line in lines:
    r, theta = line[0] # 거리 & 각도
    tx, ty = np.cos(theta), np.sin(theta)
    x0, y0 = tx*r, ty*r # x, y 기준 좌표
    # 기준 좌표에 red dot
    cv2.circle(img2, (int(abs(x0)), int(abs(y0))), 3, (0, 0, 255), -1)
    # 직선 방정식으로 그리기 위한 시작점, 끝점 계산
    x1, y1 = int(x0 + w*(-ty)), int(y0 + h*tx)
    x2, y2 = int(x0 - w*(-ty)), int(y0 - h*tx)
    # line 그리기
    cv2.line(img2, (x1, y1), (x2, y2), (0, 255, 0), 1)

# result
merged = np.hstack((img1, img2))
cv2.imshow('hough line', merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 7.2.2 확률적 허프선 변환
- lines = cv2.HoughLinesP(img, rho, theta, threshold[, lines, minLineLength, maxLineGap])
    - minLineLength : 선으로 인정할 최소길이
    - maxLineGap : 선으로 판단한 최대 간격
    - lines : 검출된 선 좌표 (N x 1 x 4) 배열 (x1, y1, x2, y2)
    - 이외는 cv2.HoughLines()와 동일

In [29]:
import cv2
import numpy as np

img1 = cv2.imread('img/sudoku.jpg')
img2 = img1.copy()
h, w = img1.shape[:2]

# to gray scale
img_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(img_gray, 50, 200)

# find Hough line
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 10, None, 20, 2)
# print(lines)

for line in lines:
    # 검출된 선 그리기
    x1, y1, x2, y2 = line[0]
    cv2.line(img1, (x1, y1), (x2, y2), (0, 255, 0), 1)
    
# result
cv2.imshow('Probability hough line', img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 7.2.3 허프 원 변환
- circles = cv2.HoughCircles(image, method, dp, minDist[, circles, param1, param2, minRadius, maxRadius])
    - method : 검출 방식 (현재 cv2.HOUGH_GRADIENT만 가능)
    - dp : 입력 영상과 경사 누적의 해상도 반비례율, 1: 입력과 동일, 값이 클수록 부정확
    - minDist : 원들 중심 간의 최소 거리, 0: error(동심원 검출 불가)
    - circles : 검출 결과 (N x 1 x 3 부동 소수점 배열, (x, y, radius))
    - param1 : 캐니 엣지에 전달할 스레시홀드 최대값
    - param2 : 경사도 누적 경계값 (값이 작을 수록 잘못된 원 검출)
    - minRadius, maxRadius: 원의 최소/최대 반지름 (0인 경우 영상의 크기)

In [39]:
import cv2
import numpy as np

img1 = cv2.imread('img/coins_connected.jpg')

# to gray scale
img_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# 노이즈 제거를 위한 가우시안 블러
blur = cv2.GaussianBlur(img_gray, (3, 3), 0)

# find Hough circles
circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1.2, 30, None, 200)

if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        # 원 둘레에 green line
        cv2.circle(img1, (i[0], i[1]), i[2], (0, 255, 0), 2)
        # center pt : red circle
        cv2.circle(img1, (i[0], i[1]), 2, (0, 0, 255), 2)
    
# result
cv2.imshow('hough circle', img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 7.3.1 거리 변환
- cv2.distanceTransform(img, distanceType, maskSize)
    - distanceType : 거리 계산 방식
        - cv2.DIST_L2, cv2.DIST_L1, cv2.DIST_L12, cv2.DIST_FAIR, cv2.DIST_WELSCH, cv2.DIST_HUBER
    - maskSize : 거리 변환 커널 크기

In [52]:
import cv2
import numpy as np

img = cv2.imread('img/full_body.jpg', cv2.IMREAD_GRAYSCALE)
_, biimg = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

# 거리 변환
dst = cv2.distanceTransform(biimg, cv2.DIST_L2, 5)
# 거리 값을 0~255로 정규화
dst = (dst/(dst.max()-dst.min())*255).astype(np.uint8) # 거리가 너무 커지지 않게 방지
# 거리 값에 스레시홀드로 완전한 뼈대 찾기
skeleton = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 7, -3)

# result
cv2.imshow('origin', img)
cv2.imshow('dist', dst)
cv2.imshow('skeleton', skeleton)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 7.3.2 연결 요소 레이블링
- 바이너리 스케일 이미지에서 픽셀 값이 0으로 끊어지지 않는 영역끼리 같은 값을 부여해서 분리하는 방법
- retval, labels = cv2.connectedComponents(image[, labels, connectivity=8, ltype]) : 연결 요소 레이블링과 개수 반환
    - labels : 레이블링된 입력 영상과 같은 크기의 배열
    - connectivity : 연결성을 검사할 방향 개수 (4 or 8)
    - ltype : 결과 레이블 배열 dtype
    - retval : 레이블 개수

- retval, labels, stats, centroids = cv2.connectedComponentsWithStats(src[, labels, stats, centroids, connectivity, ltype]) : 레이블링과 각종 상태 정보 반환
    - stats : N x 5 행렬 (N : 레이블 개수)
        - [x, y, 폭, 높이, 넓이]
    - centroids : 각 레이블의 중심점 좌표, N x 2 (N : 레이블 개수)

In [56]:
import cv2
import numpy as np

img1 = cv2.imread('img/shapes_donut.png')
img2 = np.zeros_like(img1)

# gray scale
gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
_, th = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# 연결된 요소 레이블링 적용
cnt, labels = cv2.connectedComponents(th)

# 레이블 개수만큼 순회
for i in range(cnt):
    # 레이블이 같은 영역에 랜덤한 색상 적용
    img2[labels==i] = [int(j) for j in np.random.randint(0, 255, 3)]
    
# result
cv2.imshow('origin', img1)
cv2.imshow('labeled', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 7.3.3 색 채우기
- 연속되는 영역에 같은 색상 채워넣기
- retval, img, mask, rect = cv2.floodFill(img, mask, seed, newVal[, loDiff, upDiff, flags])
    - mask : 입력 영상보다 2x2 픽셀이 더 큰 배열, 0이 아닌 영역을 만나면 채우기 중지
    - seed : 채우기 시작할 좌표
    - newVal : 채우기에 사용할 색상 값
    - loDiff, upDiff : 채우기 진행을 결정할 최소/최대 차이 값
    - flags : 채우기 방식 선택 플래그 (4 or 8 방향 채우기)
        - cv2.FLOODFILL_MASK_ONLY : img가 아닌 mask에만 채우기 적용
        - cv2.FLOODFILL_FIXED_RANGE : 이웃 픽셀이 아닌 seed 픽셀과 비교
    - retval : 채우기 한 픽셀의 개수
    - rect : 채우기가 이뤄진 영역을 감싸는 사각형
<br>
- src(x', y') - loDiff <= src(x, y) <= src(x', y') + upDiff
    - src(x, y) : 현재 픽셀
    - src(x', y') : 이웃 픽셀


In [62]:
import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')
rows, cols = img.shape[:2]

# mask & 기존 이미지 보다 2픽셀 크게
mask = np.zeros((rows+2, cols+2), np.uint8)
# 채우기에 사용할 색
newVal = (0, 0, 255)
# 최대, 최소 차이값
loDiff, upDiff = (10, 10, 10), (10, 10, 10)

# mouse event
def onMouse(event, x, y, flags, param):
    global mask, img
    if event == cv2.EVENT_LBUTTONDOWN:
        seed = (x, y)
        retval = cv2.floodFill(img, mask, seed, loDiff, upDiff) #, cv2.FLOODFILL_MASK_ONLY)
        cv2.imshow('img', img)

# result
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()